## Домашнее задание 7.

1. Попробуйте обучить нейронную сеть GRU/LSTM для предсказания сентимента сообщений с твитера на примере https://www.kaggle.com/datasets/arkhoshghalb/twitter-sentiment-analysis-hatred-speech

2. Опишите, какой результат вы получили? Что помогло вам улучшить ее точность?

## Решение

Импортируем библиотеки, которые будут необходимы при дальнейшем решении задачи.

In [1]:
import torch
import re
import pandas as pd
import numpy as np
import nltk

import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
from string import punctuation
from textblob import TextBlob, Word
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from itertools import islice
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

nltk.download("punkt")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Проверка на наличие видеокарты.

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

Зададим ряд гиперпараметров, которые будут использоваться в дальнейшем процессе обучения.

In [3]:
max_words = 1500
max_len = 15
num_classes = 1
batch_size = 512

Считываем тренировочный датасет, на основе которого будем обучать нейросеть.

In [4]:
DATA_ROOT = '/kaggle/input/twitter-sentiment-analysis-hatred-speech'

df_train = pd.read_csv(DATA_ROOT + "/train.csv")
df_train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


В данном случае мы имеем дело с несбалансированным датасетом.

In [5]:
df_train['label'].value_counts()

0    29720
1     2242
Name: label, dtype: int64

Сразу разделим данные на обучающий и валидационный наборы, добавим стратификацию, чтобы сохранить пропорции классов.

In [6]:
X_train, X_val, y_train, y_val = train_test_split(df_train['tweet'], 
                                                  df_train['label'], 
                                                  test_size=0.3, 
                                                  random_state=42, 
                                                  stratify=df_train['label'])

Теперь перейдем к предобработке твитов. Для начала сформируем множество стоп слов.

In [7]:
sw = set(stopwords.words("english"))
# Добавим к стандартному множеству еще одно слово, которое не несет смысловой нагрузки,
# но часто встречается как текстовое представление символа - &amp; 
sw.add('amp')
# Добавим user, так как в данном датасете это является обезличенным 
# упоминанием пользователя в твите
sw.add('user')
sw

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'amp',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'o

Сформируем дополнительно список знаков пунктуации.

In [8]:
puncts = set(punctuation)
puncts

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~'}

Напишем функцию по аналогии с представленной на лекции, которая будет производить предобработку подаваемого текста.

In [9]:
def preprocess_text(txt):
    txt = str(txt)
    # уберем нечитаемые символы типа  ð\x9f¤\x97
    txt = "".join([c for c in txt if ord(c) < 128])
    txt = "".join(c for c in txt if c not in puncts)
    txt = txt.lower()
    # преобразуем отрицания
    txt = re.sub("not\s", "not", txt)
    txt = re.sub("no\s", "no", txt)
    # будем приводить формы к глаголам
    txt = [Word(word).lemmatize('v') for word in txt.split() if word not in sw]
    return " ".join(txt)

Посмотрим на примерах на результат работы предобработки.

In [10]:
X_train.iloc[:10].values

array(['happy bihday to my brother man. needed this mixtape like we need boos. have a good one sach   @user ',
       '  lang to sta the week right :)  #happiness #smile ',
       'note it meditate on it work on it ,but most impoantly trust god for it #icantwaitfohedayhisplansformylifeunfold #grateful  ',
       '@user listening to you this wet mon, ahead of #leedsmillenium gig next month   ð\x9f\x98\x86ð\x9f\x91\x8dð\x9f\x98\x8d #music #ace ',
       '@user @user agreed.. the same is true for  and .. they are overused terms, and as a result, are fast becominâ\x80¦',
       'very exciting! #dubllife #recycle ',
       '#bad times #drink   #nobev ',
       '#ootd #converse #denim #tshi  #shopping  #like4like #l4l #f4f #instagood  ',
       '  #fathersday to the man of my dreams! you sacrificed bachelorhood for a ready-made familyâ\x80¦ ',
       '  #pougalday #pay #saturday #fresh #new #haircut &amp; new #red #car in #style #chillingâ\x80¦ '],
      dtype=object)

In [11]:
X_train.iloc[:10].apply(preprocess_text).values

array(['happy bihday brother man need mixtape like need boo good one sach',
       'lang sta week right happiness smile',
       'note meditate work impoantly trust god icantwaitfohedayhisplansformylifeunfold grateful',
       'listen wet mon ahead leedsmillenium gig next month music ace',
       'agree true overuse term result fast becomin',
       'excite dubllife recycle', 'bad time drink nobev',
       'ootd converse denim tshi shop like4like l4l f4f instagood',
       'fathersday man dream sacrifice bachelorhood readymade family',
       'pougalday pay saturday fresh new haircut new red car style chill'],
      dtype=object)

Теперь преобразуем тексты всех твитов с помощью данной функции.

In [12]:
X_train = X_train.apply(preprocess_text).values
X_val = X_val.apply(preprocess_text).values

Перейдем к реализации процесса токенизации.

In [13]:
train_corpus = " ".join(X_train)
train_corpus = train_corpus.lower()

In [14]:
tokens = word_tokenize(train_corpus)
tokens[:5]

['happy', 'bihday', 'brother', 'man', 'need']

In [15]:
tokens_filtered = [word for word in tokens if word.isalnum()]
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

# Посмотрим на топ 10 слов
tokens_filtered_top[:10]

['love', 'day', 'get', 'happy', 'go', 'time', 'make', 'im', 'u', 'life']

Сформируем словарь, в котором будут храниться наиболее часто встречающиеся слова.

In [16]:
def take(n, iterable):
    return list(islice(iterable, n))

vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
take(20, vocabulary.items())

[('love', 1),
 ('day', 2),
 ('get', 3),
 ('happy', 4),
 ('go', 5),
 ('time', 6),
 ('make', 7),
 ('im', 8),
 ('u', 9),
 ('life', 10),
 ('like', 11),
 ('today', 12),
 ('new', 13),
 ('father', 14),
 ('see', 15),
 ('positive', 16),
 ('smile', 17),
 ('thankful', 18),
 ('people', 19),
 ('bihday', 20)]

Запишем функцию преобразования текста в токены.

In [17]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])

    padding = [0] * (maxlen-len(result))
    return result[-maxlen:] + padding

In [18]:
%%time
x_train = np.asarray([text_to_sequence(text, max_len) for text in X_train])
x_val = np.asarray([text_to_sequence(text, max_len) for text in X_val])

CPU times: user 5.41 s, sys: 10.1 ms, total: 5.42 s
Wall time: 5.44 s


In [19]:
x_train[1]

array([165,  69,  76,  77,  17,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0])

Соберем сеть.

In [20]:
class LSTMFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, drop_prob=0.1, use_last=True):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True, dropout=drop_prob)
        self.linear = nn.Linear(hidden_dim, 1)
#         self.dropout = nn.Dropout(drop_prob)
        
    def forward(self, x):
        x = self.embeddings(x)
#         x = self.dropout(x)
        lstm_out, ht = self.lstm(x)
       
        if self.use_last:
            last_tensor = lstm_out[:,-1,:]
        else:
            # use mean
            last_tensor = torch.mean(lstm_out[:,:], dim=1)
    
        out = self.linear(last_tensor)
        return torch.sigmoid(out)

Создадим класс датасета и определим даталоадеры.

In [21]:
class DataWrapper(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).long()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
            
        return x, y
    
    def __len__(self):
        return len(self.data)

In [22]:
train_dataset = DataWrapper(x_train, y_train.values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = DataWrapper(x_val, y_val.values)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

Задаем критерий с учетом бинарности результатов.

In [23]:
criterion = nn.BCELoss()

Проведем перебор по сетке для определения оптимальных гиперпараметров модели LSTM.

In [24]:
n_epochs = [5, 10]
learning_rates = [1e-2, 1e-3]
e_dims = [128, 256]
h_dims = [64, 96]
ths = [0.3, 0.5]
dps = [0.1, 0.2, 0.3]

In [25]:
for epochs in n_epochs:
    for lr in learning_rates:
        for embedding_dim in e_dims:
            for hidden_dim in h_dims:
                for th in ths:
                    for dp in dps:
                        
                        print(f'Hyper params: epochs - {epochs}, learning_rate - {lr}, '
                             f'embedding_dim - {embedding_dim}, hidden_dim - {hidden_dim}, '
                             f'threshold_level - {th}, drop_prob - {dp}.')
                        model = LSTMFixedLen(vocab_size=max_words, 
                                             embedding_dim=embedding_dim, hidden_dim=hidden_dim, 
                                             drop_prob=dp, use_last=False)
                        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
                        model = model.to(device)
                        model.train()
                        th = th

                        train_loss_history = []
                        test_loss_history = []


                        for epoch in range(epochs):  
                            running_items, running_right = 0.0, 0.0
                            for i, data in enumerate(train_loader, 0):
                                inputs, labels = data[0].to(device), data[1].to(device)

                                # обнуляем градиент
                                optimizer.zero_grad()
                                outputs = model(inputs)

                                loss = criterion(outputs, labels.float().view(-1, 1))
                                loss.backward()
                                optimizer.step()

                                # подсчет ошибки на обучении
                                loss = loss.item()
                                running_items += len(labels)
                                # подсчет метрики на обучении
                                pred_labels = torch.squeeze((outputs > th).int())
                                running_right += (labels == pred_labels).sum()

                            # выводим статистику о процессе обучения
                            model.eval()

                            print(f'Epoch [{epoch + 1}/{epochs}]. ' \
                                    f'Step [{i + 1}/{len(train_loader)}]. ' \
                                    f'Loss: {loss:.3f}. ' \
                                    f'Acc: {running_right / running_items:.3f}', end='. ')
                            running_loss, running_items, running_right = 0.0, 0.0, 0.0
                            train_loss_history.append(loss)

                                # выводим статистику на тестовых данных
                            test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
                            for j, data in enumerate(val_loader):
                                test_labels = data[1].to(device)
                                test_outputs = model(data[0].to(device))

                                # подсчет ошибки на тесте
                                test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
                                # подсчет метрики на тесте
                                test_running_total += len(data[1])
                                pred_test_labels = torch.squeeze((test_outputs > th).int())
                                test_running_right += (test_labels == pred_test_labels).sum()

                            test_loss_history.append(test_loss.item())
                            print(f'Test loss: {test_loss:.3f}.' 
                                  f'Test acc: {test_running_right / test_running_total:.3f}')

                            model.train()

                        print('Training is finished!')

Hyper params: epochs - 5, learning_rate - 0.01, embedding_dim - 128, hidden_dim - 64, threshold_level - 0.3, drop_prob - 0.1.
Epoch [1/5]. Step [44/44]. Loss: 0.202. Acc: 0.875. Test loss: 0.454.Test acc: 0.941
Epoch [2/5]. Step [44/44]. Loss: 0.145. Acc: 0.946. Test loss: 0.573.Test acc: 0.937
Epoch [3/5]. Step [44/44]. Loss: 0.105. Acc: 0.955. Test loss: 0.006.Test acc: 0.942
Epoch [4/5]. Step [44/44]. Loss: 0.078. Acc: 0.965. Test loss: 0.003.Test acc: 0.948
Epoch [5/5]. Step [44/44]. Loss: 0.063. Acc: 0.969. Test loss: 0.010.Test acc: 0.950
Training is finished!
Hyper params: epochs - 5, learning_rate - 0.01, embedding_dim - 128, hidden_dim - 64, threshold_level - 0.3, drop_prob - 0.2.
Epoch [1/5]. Step [44/44]. Loss: 0.155. Acc: 0.891. Test loss: 0.119.Test acc: 0.937
Epoch [2/5]. Step [44/44]. Loss: 0.138. Acc: 0.948. Test loss: 0.039.Test acc: 0.941
Epoch [3/5]. Step [44/44]. Loss: 0.108. Acc: 0.956. Test loss: 0.027.Test acc: 0.940
Epoch [4/5]. Step [44/44]. Loss: 0.106. Acc: 0

На основе проведенного анализа можно утверждать, что уменьшение порога отсечения не позволяет добиться улучшения модели, поэтому лучше оставить этот показатель на уровне 0.5. Аналогично не дает сколько либо значимых улучшений увеличиние величины дропаута. Самым эффективными по оказываемому влиянию оказываются уменьшение скорости обучения и изменение размерности слоев.

Проведем аналогичный поиск по сетке для модели GRU.

In [26]:
class GRUFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, drop_prob=0.1, use_last=True):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=2, batch_first=True, dropout=drop_prob)
        self.linear = nn.Linear(hidden_dim, 1)
#         self.dropout = nn.Dropout(drop_prob)
        
    def forward(self, x):
        x = self.embeddings(x)
#         x = self.dropout(x)
        gru_out, ht = self.gru(x)
       
        if self.use_last:
            last_tensor = gru_out[:,-1,:]
        else:
            # use mean
            last_tensor = torch.mean(gru_out[:,:], dim=1)
    
        out = self.linear(last_tensor)
        return torch.sigmoid(out)

In [27]:
for epochs in n_epochs:
    for lr in learning_rates:
        for embedding_dim in e_dims:
            for hidden_dim in h_dims:
                for th in ths:
                    for dp in dps:
                        
                        print(f'Hyper params: epochs - {epochs}, learning_rate - {lr}, '
                             f'embedding_dim - {embedding_dim}, hidden_dim - {hidden_dim}, '
                             f'threshold_level - {th}, drop_prob - {dp}.')
                        model = GRUFixedLen(vocab_size=max_words, 
                                             embedding_dim=embedding_dim, hidden_dim=hidden_dim, 
                                             drop_prob=dp, use_last=False)
                        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
                        model = model.to(device)
                        model.train()
                        th = th

                        train_loss_history = []
                        test_loss_history = []


                        for epoch in range(epochs):  
                            running_items, running_right = 0.0, 0.0
                            for i, data in enumerate(train_loader, 0):
                                inputs, labels = data[0].to(device), data[1].to(device)

                                # обнуляем градиент
                                optimizer.zero_grad()
                                outputs = model(inputs)

                                loss = criterion(outputs, labels.float().view(-1, 1))
                                loss.backward()
                                optimizer.step()

                                # подсчет ошибки на обучении
                                loss = loss.item()
                                running_items += len(labels)
                                # подсчет метрики на обучении
                                pred_labels = torch.squeeze((outputs > th).int())
                                running_right += (labels == pred_labels).sum()

                            # выводим статистику о процессе обучения
                            model.eval()

                            print(f'Epoch [{epoch + 1}/{epochs}]. ' \
                                    f'Step [{i + 1}/{len(train_loader)}]. ' \
                                    f'Loss: {loss:.3f}. ' \
                                    f'Acc: {running_right / running_items:.3f}', end='. ')
                            running_loss, running_items, running_right = 0.0, 0.0, 0.0
                            train_loss_history.append(loss)

                                # выводим статистику на тестовых данных
                            test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
                            for j, data in enumerate(val_loader):
                                test_labels = data[1].to(device)
                                test_outputs = model(data[0].to(device))

                                # подсчет ошибки на тесте
                                test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
                                # подсчет метрики на тесте
                                test_running_total += len(data[1])
                                pred_test_labels = torch.squeeze((test_outputs > th).int())
                                test_running_right += (test_labels == pred_test_labels).sum()

                            test_loss_history.append(test_loss.item())
                            print(f'Test loss: {test_loss:.3f}.' 
                                  f'Test acc: {test_running_right / test_running_total:.3f}')

                            model.train()

                        print('Training is finished!')

Hyper params: epochs - 5, learning_rate - 0.01, embedding_dim - 128, hidden_dim - 64, threshold_level - 0.3, drop_prob - 0.1.
Epoch [1/5]. Step [44/44]. Loss: 0.164. Acc: 0.896. Test loss: 0.026.Test acc: 0.947
Epoch [2/5]. Step [44/44]. Loss: 0.105. Acc: 0.952. Test loss: 0.015.Test acc: 0.945
Epoch [3/5]. Step [44/44]. Loss: 0.133. Acc: 0.960. Test loss: 0.023.Test acc: 0.940
Epoch [4/5]. Step [44/44]. Loss: 0.114. Acc: 0.969. Test loss: 0.038.Test acc: 0.948
Epoch [5/5]. Step [44/44]. Loss: 0.074. Acc: 0.978. Test loss: 0.012.Test acc: 0.937
Training is finished!
Hyper params: epochs - 5, learning_rate - 0.01, embedding_dim - 128, hidden_dim - 64, threshold_level - 0.3, drop_prob - 0.2.
Epoch [1/5]. Step [44/44]. Loss: 0.162. Acc: 0.899. Test loss: 0.022.Test acc: 0.941
Epoch [2/5]. Step [44/44]. Loss: 0.157. Acc: 0.951. Test loss: 0.024.Test acc: 0.949
Epoch [3/5]. Step [44/44]. Loss: 0.083. Acc: 0.960. Test loss: 0.023.Test acc: 0.946
Epoch [4/5]. Step [44/44]. Loss: 0.067. Acc: 0

Видим, что данная модель на пяти эпохах показывает в целом сопоставимые результаты, на десяти эпохах показатели несколько хуже, чем у LSTM, но разница все равно довольно незначительная.

Обучаем итоговую модель.

In [28]:
model = LSTMFixedLen(vocab_size=max_words, 
                 embedding_dim=256, hidden_dim=96, 
                 drop_prob=0.1, use_last=False)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [29]:
print(model)
print("Parameters:", sum([param.nelement() for param in model.parameters()]))

LSTMFixedLen(
  (embeddings): Embedding(1500, 256, padding_idx=0)
  (lstm): LSTM(256, 96, num_layers=2, batch_first=True, dropout=0.1)
  (linear): Linear(in_features=96, out_features=1, bias=True)
)
Parameters: 594529


In [30]:
model = model.to(device)
model.train()
th = 0.5
epochs = 5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    running_items, running_right = 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        # обнуляем градиент
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()
        running_items += len(labels)
        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()

    # выводим статистику о процессе обучения
    model.eval()

    print(f'Epoch [{epoch + 1}/{epochs}]. ' \
            f'Step [{i + 1}/{len(train_loader)}]. ' \
            f'Loss: {loss:.3f}. ' \
            f'Acc: {running_right / running_items:.3f}', end='. ')
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    train_loss_history.append(loss)

        # выводим статистику на тестовых данных
    test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
    for j, data in enumerate(val_loader):
        test_labels = data[1].to(device)
        test_outputs = model(data[0].to(device))

        # подсчет ошибки на тесте
        test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
        # подсчет метрики на тесте
        test_running_total += len(data[1])
        pred_test_labels = torch.squeeze((test_outputs > th).int())
        test_running_right += (test_labels == pred_test_labels).sum()

    test_loss_history.append(test_loss.item())
    print(f'Test loss: {test_loss:.3f}.' 
          f'Test acc: {test_running_right / test_running_total:.3f}')

    model.train()

print('Training is finished!')

Epoch [1/5]. Step [44/44]. Loss: 0.212. Acc: 0.867. Test loss: 0.218.Test acc: 0.936
Epoch [2/5]. Step [44/44]. Loss: 0.213. Acc: 0.945. Test loss: 0.540.Test acc: 0.946
Epoch [3/5]. Step [44/44]. Loss: 0.123. Acc: 0.956. Test loss: 0.041.Test acc: 0.951
Epoch [4/5]. Step [44/44]. Loss: 0.079. Acc: 0.963. Test loss: 0.473.Test acc: 0.953
Epoch [5/5]. Step [44/44]. Loss: 0.178. Acc: 0.970. Test loss: 0.025.Test acc: 0.950
Training is finished!


В целом результаты обучения модели получилось довольно схожими с теми, что были получены при построении сети с применением одномерных сверток. Самыми эффективными гиперпараметрами для улучшения качества модели оказались скорость обучения (ее уменьшение) и размерность слоев: эмбеддинга и скрытого (их увеличение).